# Setting Up the Database 

#### Importing packages 

In [1]:
import pandas as pd

## 1. Downloading the MTA CSV files from January to March, 2021*
* From week ending January 2, 2021 to week ending March 27, 2021

In [2]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week_num in week_nums: #for each week
        file_url = url.format(week_num) #format the url so it grabs that week
        dfs.append(pd.read_csv(file_url)) #append to our empty list
    return pd.concat(dfs) #combine files for each week
        
week_nums = [210102, 210109, 210116, 210123, 210130, 210206, 210213, 210220, 210227, 210306, 210313, 210320, 210327]
turnstiles_df = get_data(week_nums) #use our function to get a df with all of our data

#### Exporting the combined file 

In [3]:
turnstiles_df.to_csv('mta_data_all.csv', header = False, index = False)

## 2. SQL 
### 2.1 Creating an SQL table

```
    sqlite3 mta.db
    
   >sqlite CREATE TABLE mta_data (
        CA TEXT,
        UNIT TEXT,
        SCP TEXT,
        STATION TEXT,
        LINENAME TEXT,
        DIVISION TEXT,
        DATE TEXT,
        TIME TEXT,
        DESC TEXT,
        ENTRIES INTEGER,
        EXITS INTEGER,
        PRIMARY KEY (CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS)
        );
```

#### Inserting the MTA CSV file into the SQL table 

```
    >sqlite .mode csv mta_data
    >sqlite .import mta_data_all.csv mta_data;

```

#### Accessing the SQL table through Python with SQLAlchemy

In [4]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("sqlite:///mta.db")
all_data = pd.read_sql('SELECT * FROM mta_data;', engine)

### 2.2 Running some queries with SQLAlchemy

#### Looking at the top 5 rows

In [5]:
top_rows = pd.read_sql('SELECT * FROM mta_data LIMIT 5;', engine)
top_rows

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,03:00:00,REGULAR,7508509,2557569
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,07:00:00,REGULAR,7508511,2557581
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,11:00:00,REGULAR,7508531,2557636
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,15:00:00,REGULAR,7508610,2557667
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,19:00:00,REGULAR,7508767,2557689


#### Field Description from the MTA:

C/A = Control Area (A002)  UNIT = Remote Unit for a station (R051)  
SCP = Subunit Channel Position represents an specific address for a device (02-00-00)  
STATION = Represents the station name the device is located at  
LINENAME = Represents all train lines that can be boarded at this station. Normally lines are represented by one character. LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.  
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND  DATE = Represents the date (MM-DD-YY)  
TIME = Represents the time (hh:mm:ss) for a scheduled audit event  
DESC = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered.  ENTRIES = The comulative entry register value for a device  
EXITS = The cumulative exit register value for a device


#### Finding the total number of stations

In [6]:
num_stations = pd.read_sql('SELECT COUNT(DISTINCT STATION) as num_stations FROM mta_data;', engine)
num_stations

,num_stations
0,378


#### Finding null values 

In [7]:
missing_values = pd.read_sql('SELECT * FROM mta_data WHERE ENTRIES IS NULL OR EXITS IS NULL;', engine)
missing_values

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


#### Finding exits and entries of 0

In [8]:
missing_values_2 = pd.read_sql('SELECT * FROM mta_data WHERE ENTRIES = \'0\' OR EXITS = \'0\' LIMIT 5;', engine)
missing_values_2 

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-05-00,59 ST,NQR456W,BMT,12/26/2020,03:00:00,REGULAR,524397,0
1,A002,R051,02-05-00,59 ST,NQR456W,BMT,12/26/2020,07:00:00,REGULAR,524397,0
2,A002,R051,02-05-00,59 ST,NQR456W,BMT,12/26/2020,11:00:00,REGULAR,524397,0
3,A002,R051,02-05-00,59 ST,NQR456W,BMT,12/26/2020,15:00:00,REGULAR,524397,0
4,A002,R051,02-05-00,59 ST,NQR456W,BMT,12/26/2020,19:00:00,REGULAR,524397,0


Seeing if any entries are 0

In [9]:
missing_values_2 = pd.read_sql('SELECT * FROM mta_data WHERE ENTRIES = \'0\' LIMIT 5;', engine)
missing_values_2 

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-05-01,59 ST,NQR456W,BMT,12/26/2020,03:00:00,REGULAR,0,887
1,A002,R051,02-05-01,59 ST,NQR456W,BMT,12/26/2020,07:00:00,REGULAR,0,887
2,A002,R051,02-05-01,59 ST,NQR456W,BMT,12/26/2020,11:00:00,REGULAR,0,887
3,A002,R051,02-05-01,59 ST,NQR456W,BMT,12/26/2020,15:00:00,REGULAR,0,887
4,A002,R051,02-05-01,59 ST,NQR456W,BMT,12/26/2020,19:00:00,REGULAR,0,887


Counting number of exits and entries of 0

In [10]:
missing_values_2 = pd.read_sql('SELECT COUNT(*) FROM mta_data WHERE ENTRIES = \'0\' OR EXITS = \'0\';', engine)
missing_values_2 

,COUNT(*)
0,145697


This probably makes sense since sometimes noone is entering or exiting and stations often have planned works, in which case there will be no entries/exits 

#### Checking that all entries (and exits) are unique

In [11]:
unique_entries = pd.read_sql('SELECT CA, UNIT, SCP, STATION, DATE, TIME, COUNT(ENTRIES) FROM mta_data GROUP BY CA, UNIT, SCP, STATION, DATE, TIME ORDER BY COUNT(ENTRIES) DESC LIMIT 10;', engine)
unique_entries

,CA,UNIT,SCP,STATION,DATE,TIME,COUNT(ENTRIES)
0,B028,R136,01-00-01,SHEEPSHEAD BAY,01/08/2021,04:00:00,2
1,N100,R252,00-00-00,HIGH ST,03/06/2021,03:00:00,2
2,N100,R252,00-00-00,HIGH ST,03/06/2021,07:00:00,2
3,N100,R252,00-00-00,HIGH ST,03/06/2021,11:00:00,2
4,N100,R252,00-00-00,HIGH ST,03/06/2021,19:00:00,2
5,N100,R252,00-00-00,HIGH ST,03/06/2021,23:00:00,2
6,N100,R252,00-00-00,HIGH ST,03/07/2021,03:00:00,2
7,N100,R252,00-00-00,HIGH ST,03/07/2021,07:00:00,2
8,N100,R252,00-00-00,HIGH ST,03/07/2021,11:00:00,2
9,N100,R252,00-00-00,HIGH ST,03/07/2021,15:00:00,2


It seems that quite a few entries (and exits) are not unique... 

#### Finding how many entries are not unique 

Finding the total number of rows

In [12]:
row_num = pd.read_sql('SELECT COUNT(*) FROM mta_data;', engine)
row_num

,COUNT(*)
0,2719827


Finding the number of duplicate rows 

In [13]:
duplicate_entries_num = pd.read_sql('SELECT COUNT(*) FROM (SELECT COUNT(*) FROM mta_data GROUP BY CA, UNIT, SCP, STATION, DATE, TIME HAVING COUNT(ENTRIES) = 2);', engine)
duplicate_entries_num

,COUNT(*)
0,75


#### Having a look at one of the duplicate entries

In [14]:
duplicate_sample = pd.read_sql('SELECT * FROM mta_data WHERE CA=\'B028\' AND UNIT=\'R136\' AND SCP=\'01-00-01\' AND STATION=\'SHEEPSHEAD BAY\' AND DATE=\'01/08/2021\' AND TIME=\'04:00:00\';', engine)
duplicate_sample

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,B028,R136,01-00-01,SHEEPSHEAD BAY,BQ,BMT,01/08/2021,04:00:00,RECOVR AUD,4204109,99386
1,B028,R136,01-00-01,SHEEPSHEAD BAY,BQ,BMT,01/08/2021,04:00:00,REGULAR,271949,99386


#### Seeing if the duplicated data all share the same description of "RECOVR AUD", which refers to a missed audit that was recovered

In [15]:
duplicate_descrip = pd.read_sql('SELECT DISTINCT DESC FROM mta_data GROUP BY CA, UNIT, SCP, STATION, DATE, TIME HAVING COUNT(ENTRIES) = 2 LIMIT 10;', engine)
duplicate_descrip

,DESC
0,RECOVR AUD


## 3. Cleaning the data with Python
### 3.1 Stripping whitespaces from the columns 

In [4]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

### 3.2 Adding a datetime column

In [5]:
import datetime
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,03:00:00,REGULAR,7508509,2557569,2020-12-26 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,07:00:00,REGULAR,7508511,2557581,2020-12-26 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,11:00:00,REGULAR,7508531,2557636,2020-12-26 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,15:00:00,REGULAR,7508610,2557667,2020-12-26 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,19:00:00,REGULAR,7508767,2557689,2020-12-26 19:00:00


Just in case we still want to use the date and time columns individually...

In [6]:
turnstiles_df['DATE'] = pd.to_datetime(turnstiles_df.DATE, format="%m/%d/%Y")
turnstiles_df['TIME'] = pd.to_datetime(turnstiles_df.TIME, format="%H:%M:%S")

### 3.3 Dealing with duplicate entries/exits 
Removing the recovery audit rows (do not run again)

In [7]:
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=False)

turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

Checking that the duplicate rows have been removed

In [8]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2020-12-26 03:00:00,1
1813161,R138,R293,00-03-04,34 ST-PENN STA,2020-12-26 05:00:00,1
1813163,R138,R293,00-03-04,34 ST-PENN STA,2020-12-26 13:00:00,1
1813164,R138,R293,00-03-04,34 ST-PENN STA,2020-12-26 17:00:00,1
1813165,R138,R293,00-03-04,34 ST-PENN STA,2020-12-26 21:00:00,1


Dropping the description column  
To prevent errors in multiple run of cell, errors on drop is ignored

In [9]:
turnstiles_df = turnstiles_df.drop("DESC", axis=1, errors="ignore")

### 3.4 Creating a column for the sum of entries and exits 

In [10]:
turnstiles_df["ENTRIES_EXITS"] = turnstiles_df["ENTRIES"] + turnstiles_df["EXITS"]
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATE_TIME,ENTRIES_EXITS
209469,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 21:00:00,5554,544,2021-03-26 21:00:00,6098
209468,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 17:00:00,5554,544,2021-03-26 17:00:00,6098
209467,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 13:00:00,5554,544,2021-03-26 13:00:00,6098
209466,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 09:00:00,5554,544,2021-03-26 09:00:00,6098
209465,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 05:00:00,5554,544,2021-03-26 05:00:00,6098


#### Dropping the entries and exits columns  
To prevent errors in multiple run of cell, errors on drop is ignored

In [12]:
turnstiles_df = turnstiles_df.drop(["ENTRIES", "EXITS"], axis=1, errors="ignore")
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DATE_TIME,ENTRIES_EXITS
209469,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 21:00:00,2021-03-26 21:00:00,6098
209468,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 17:00:00,2021-03-26 17:00:00,6098
209467,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 13:00:00,2021-03-26 13:00:00,6098
209466,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 09:00:00,2021-03-26 09:00:00,6098
209465,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 05:00:00,2021-03-26 05:00:00,6098


#### Looking at the dataframe before pickling

In [13]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DATE_TIME,ENTRIES_EXITS
209469,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 21:00:00,2021-03-26 21:00:00,6098
209468,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 17:00:00,2021-03-26 17:00:00,6098
209467,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 13:00:00,2021-03-26 13:00:00,6098
209466,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 09:00:00,2021-03-26 09:00:00,6098
209465,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-03-26,1900-01-01 05:00:00,2021-03-26 05:00:00,6098


# 4. Pickling the dataframe for exploratory analysis

In [14]:
import pickle
turnstiles_df.to_pickle('turnstiles_df.pickle')